In [1]:
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [2]:
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

In [3]:
#pre-process all the train images
train_generator = train_data_gen.flow_from_directory('./DataSet/images/train',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 25311 images belonging to 7 classes.


In [13]:
#pre-process all the test images
validation_generator = validation_data_gen.flow_from_directory('./DataSet/images/validation',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 6911 images belonging to 7 classes.


In [14]:
# create model structure
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

cv2.ocl.setUseOpenCL(False)

emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001, decay=1e-6), metrics=['accuracy'])


In [15]:
# Train the neural network/model
emotion_model_info = emotion_model.fit_generator(
        train_generator,
        steps_per_epoch=25311 // 64,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=6911 // 64)


C:\Users\HP\AppData\Local\Temp\ipykernel_2428\3783702684.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model_info = emotion_model.fit_generator(


Epoch 1/50
395/395 [==============================] - 695s 2s/step - loss: 1.8012 - accuracy: 0.2734 - val_loss: 1.7512 - val_accuracy: 0.2823
Epoch 2/50
395/395 [==============================] - 252s 638ms/step - loss: 1.6257 - accuracy: 0.3704 - val_loss: 1.5712 - val_accuracy: 0.3968
Epoch 3/50
395/395 [==============================] - 64s 163ms/step - loss: 1.5193 - accuracy: 0.4200 - val_loss: 1.5000 - val_accuracy: 0.4387
Epoch 4/50
395/395 [==============================] - 64s 163ms/step - loss: 1.4504 - accuracy: 0.4483 - val_loss: 1.4463 - val_accuracy: 0.4445
Epoch 5/50
395/395 [==============================] - 64s 163ms/step - loss: 1.3949 - accuracy: 0.4733 - val_loss: 1.4014 - val_accuracy: 0.4658
Epoch 6/50
395/395 [==============================] - 64s 162ms/step - loss: 1.3466 - accuracy: 0.4944 - val_loss: 1.3587 - val_accuracy: 0.4784
Epoch 7/50
395/395 [==============================] - 64s 162ms/step - loss: 1.2953 - accuracy: 0.5139 - val_loss: 1.3172 - val_acc

In [16]:
# save model structure in jason file
model_json = emotion_model.to_json()
with open("emotion_model.json", "w") as json_file:
    json_file.write(model_json)

# save trained model weight in .h5 file
emotion_model.save_weights('emotion_model.h5')

